In [6]:
import pandas as pd
import numpy as np
import re

In [7]:
data = pd.read_csv('bbc-news-data.csv' , sep='\t')

In [8]:
data.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [9]:
data.rename(columns={'category': 'Label'}, inplace=True)
data.rename(columns={'title': 'Headline'}, inplace=True)
data.rename(columns={'content': 'Description'}, inplace=True)
data.drop(columns=['filename'], inplace=True)
data.head()

,Label,Headline,Description
0,business,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [10]:
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

def clean_description(description):
    description = description.lower()
    description = description.translate(str.maketrans('', '', string.punctuation))
    description = description.strip()

    tokens = word_tokenize(description)

    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [word for word in tokens if word not in stop_words]

    cleaned_description = ' '.join(cleaned_tokens)

    return cleaned_description
data['Description'] = data['Description'].apply(clean_description)
data['Headline'] = data['Headline'].apply(clean_description)


In [13]:
data.head()

,Label,Headline,Description
0,business,ad sales boost time warner profit,quarterly profits us media giant timewarner ju...
1,business,dollar gains greenspan speech,dollar hit highest level euro almost three mon...
2,business,yukos unit buyer faces loan claim,owners embattled russian oil giant yukos ask b...
3,business,high fuel prices hit bas profits,british airways blamed high fuel prices 40 dro...
4,business,pernod takeover talk lifts domecq,shares uk drinks food firm allied domecq risen...


In [15]:
rows_to_drop = data[data['Label'] == 'entertainment'].index
data = data.drop(rows_to_drop)

In [16]:
data.to_csv('./data.csv', index=False)


In [17]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [18]:
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['Headline'] + ' ' + train_data['Description'])

X_train = tokenizer.texts_to_sequences(train_data['Headline'] + ' ' + train_data['Description'])
X_val = tokenizer.texts_to_sequences(val_data['Headline'] + ' ' + val_data['Description'])
X_test = tokenizer.texts_to_sequences(test_data['Headline'] + ' ' + test_data['Description'])

max_length = 1000  # مقدار دلخواه برای طول دنباله‌ها
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

label_dict = {label: idx for idx, label in enumerate(train_data['Label'].unique())}
y_train = np.array([label_dict[label] for label in train_data['Label']])
y_val = np.array([label_dict[label] for label in val_data['Label']])
y_test = np.array([label_dict[label] for label in test_data['Label']])


In [20]:
# RNN
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_length))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=len(label_dict), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=100)


Epoch 1/50
13/13 [==============================] - 45s 3s/step - loss: 1.3854 - accuracy: 0.2774 - val_loss: 1.3812 - val_accuracy: 0.2500
Epoch 2/50
13/13 [==============================] - 39s 3s/step - loss: 1.3817 - accuracy: 0.2611 - val_loss: 1.3805 - val_accuracy: 0.2500
Epoch 3/50
13/13 [==============================] - 38s 3s/step - loss: 1.3809 - accuracy: 0.2743 - val_loss: 1.3811 - val_accuracy: 0.2500
Epoch 4/50
13/13 [==============================] - 38s 3s/step - loss: 1.3817 - accuracy: 0.2797 - val_loss: 1.3837 - val_accuracy: 0.2500
Epoch 5/50
13/13 [==============================] - 36s 3s/step - loss: 1.3808 - accuracy: 0.2821 - val_loss: 1.3814 - val_accuracy: 0.2500
Epoch 6/50
13/13 [==============================] - 37s 3s/step - loss: 1.3810 - accuracy: 0.2751 - val_loss: 1.3804 - val_accuracy: 0.3043
Epoch 7/50
13/13 [==============================] - 37s 3s/step - loss: 1.3809 - accuracy: 0.2758 - val_loss: 1.3813 - val_accuracy: 0.2500
Epoch 8/50
13/13 [==

In [21]:
from keras.optimizers import SGD

# RNN
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_length))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=len(label_dict), activation='softmax'))

# SGD
optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

Epoch 1/10
21/21 [==============================] - 54s 2s/step - loss: 1.3850 - accuracy: 0.2681 - val_loss: 1.3819 - val_accuracy: 0.2500
Epoch 2/10
21/21 [==============================] - 50s 2s/step - loss: 1.3831 - accuracy: 0.2813 - val_loss: 1.3808 - val_accuracy: 0.3043
Epoch 3/10
21/21 [==============================] - 50s 2s/step - loss: 1.3826 - accuracy: 0.2463 - val_loss: 1.3800 - val_accuracy: 0.3043
Epoch 4/10
21/21 [==============================] - 47s 2s/step - loss: 1.3825 - accuracy: 0.2789 - val_loss: 1.3808 - val_accuracy: 0.2500
Epoch 5/10
21/21 [==============================] - 49s 2s/step - loss: 1.3841 - accuracy: 0.2813 - val_loss: 1.3827 - val_accuracy: 0.2500
Epoch 6/10
21/21 [==============================] - 48s 2s/step - loss: 1.3832 - accuracy: 0.2813 - val_loss: 1.3820 - val_accuracy: 0.2500
Epoch 7/10
21/21 [==============================] - 47s 2s/step - loss: 1.3812 - accuracy: 0.2813 - val_loss: 1.3816 - val_accuracy: 0.2500
Epoch 8/10
21/21 [==

In [19]:
from keras.optimizers import SGD

# RNN
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_length))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=len(label_dict), activation='softmax'))

optimizer = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

Epoch 1/10
25/25 [==============================] - 60s 2s/step - loss: 1.6100 - accuracy: 0.1792 - val_loss: 1.6098 - val_accuracy: 0.1946
Epoch 2/10
25/25 [==============================] - 55s 2s/step - loss: 1.6070 - accuracy: 0.2338 - val_loss: 1.6078 - val_accuracy: 0.1946
Epoch 3/10
25/25 [==============================] - 54s 2s/step - loss: 1.6048 - accuracy: 0.2389 - val_loss: 1.6070 - val_accuracy: 0.1946
Epoch 4/10
25/25 [==============================] - 57s 2s/step - loss: 1.6037 - accuracy: 0.2383 - val_loss: 1.6063 - val_accuracy: 0.1946
Epoch 5/10
25/25 [==============================] - 55s 2s/step - loss: 1.6030 - accuracy: 0.2383 - val_loss: 1.6063 - val_accuracy: 0.1946
Epoch 6/10
25/25 [==============================] - 54s 2s/step - loss: 1.6026 - accuracy: 0.2383 - val_loss: 1.6062 - val_accuracy: 0.1946
Epoch 7/10
25/25 [==============================] - 59s 2s/step - loss: 1.6019 - accuracy: 0.2383 - val_loss: 1.6062 - val_accuracy: 0.1946
Epoch 8/10
25/25 [==

In [22]:
from keras.optimizers import SGD

# RNN
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=max_length))
model.add(LSTM(units=128, dropout=0.25, recurrent_dropout=0.2))
model.add(Dense(units=len(label_dict), activation='softmax'))
# model.add(LSTM(units=64, dropout=0.3, recurrent_dropout=0.25))

# model.add(Dense(units=len(label_dict), activation='softmax'))

optimizer = SGD(learning_rate=0.001, momentum=0.95)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

Epoch 1/10
25/25 [==============================] - 105s 4s/step - loss: 1.6094 - accuracy: 0.2184 - val_loss: 1.6101 - val_accuracy: 0.1946
Epoch 2/10
25/25 [==============================] - 100s 4s/step - loss: 1.6060 - accuracy: 0.2383 - val_loss: 1.6093 - val_accuracy: 0.1946
Epoch 3/10
25/25 [==============================] - 99s 4s/step - loss: 1.6032 - accuracy: 0.2383 - val_loss: 1.6086 - val_accuracy: 0.1946
Epoch 4/10
25/25 [==============================] - 100s 4s/step - loss: 1.6020 - accuracy: 0.2383 - val_loss: 1.6068 - val_accuracy: 0.1946
Epoch 5/10
25/25 [==============================] - 100s 4s/step - loss: 1.6017 - accuracy: 0.2383 - val_loss: 1.6065 - val_accuracy: 0.1946
Epoch 6/10
25/25 [==============================] - 103s 4s/step - loss: 1.6013 - accuracy: 0.2383 - val_loss: 1.6053 - val_accuracy: 0.1946
Epoch 7/10
25/25 [==============================] - 98s 4s/step - loss: 1.6015 - accuracy: 0.2383 - val_loss: 1.6060 - val_accuracy: 0.1946
Epoch 8/10
25/2